# 02. Preprocesado de los datos

Debido a que el modelo únicamente puede entender números, debemos preparar los datos preprocesandolos para que sean los más adecuados para el modelo. Esto incluye la conversión de cadenas de texto a números, normalización de valores, rellenar datos faltantes, etc.

### Carga inicial del conjunto de datos `train.csv`

In [153]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("train.csv")
train_df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


### Eliminar columna duplicada `F_TIENEINTERNET.1`

Observamos que hay una columna `F_TIENEINTERNET` y una `F_TIENEINTERNET.1`. Tenemos que comprobar si son iguales, para eliminar una de ellas.

In [154]:
(train_df['F_TIENEINTERNET'] == train_df['F_TIENEINTERNET.1']).all()

np.False_

In [155]:
# Eliminamos la columna duplicada
train_df = train_df.drop(columns=['F_TIENEINTERNET.1'])

### Eliminar columna `ID`

Esta columna no tiene importancia en la predicción de la variable `RENDIMIENTO_GLOBAL` por lo que se decide eliminarla.

In [170]:
train_df = train_df.drop(columns=['ID'])

### Valores faltantes en las columnas

In [156]:
missing_values = train_df.isna().sum()
missing_values[missing_values!=0]

,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
F_TIENELAVADORA,39773
F_TIENEAUTOMOVIL,43623
E_PAGOMATRICULAPROPIO,6498
F_TIENECOMPUTADOR,38103
F_EDUCACIONMADRE,23664


### Rellenar datos nulos

Dada la anterior información, podemos ver que que solo hay valores nulos en las columnas de texto. Vamos a rellenar esos espacios con un valor de texto definido para esos valores nulos, como `no info`.

In [157]:
missing_values = missing_values[missing_values!=0]
for column in missing_values.index:
  train_df[column] = train_df[column].fillna('no info')

train_df.isna().sum().sum()

np.int64(0)

### Comprobar los valores únicos de las columnas con valores de texto

In [158]:
for column in train_df.select_dtypes(include="object").columns:
  print("%30s"%column, train_df[column].nunique())

              E_PRGM_ACADEMICO 948
           E_PRGM_DEPARTAMENTO 31
   E_VALORMATRICULAUNIVERSIDAD 9
          E_HORASSEMANATRABAJA 6
             F_ESTRATOVIVIENDA 8
               F_TIENEINTERNET 3
              F_EDUCACIONPADRE 13
               F_TIENELAVADORA 3
              F_TIENEAUTOMOVIL 3
            E_PRIVADO_LIBERTAD 2
         E_PAGOMATRICULAPROPIO 3
             F_TIENECOMPUTADOR 3
              F_EDUCACIONMADRE 13
            RENDIMIENTO_GLOBAL 4


In [159]:
for column in train_df.select_dtypes(include="object").drop(columns=['E_PRGM_ACADEMICO']).columns:  # Excluimos E_PRGM_ACADEMICO porque son muchos valores
  print(column, np.unique(train_df[column].dropna()))

E_PRGM_DEPARTAMENTO ['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLANTICO' 'BOGOTÁ' 'BOLIVAR' 'BOYACA'
 'CALDAS' 'CAQUETA' 'CASANARE' 'CAUCA' 'CESAR' 'CHOCO' 'CORDOBA'
 'CUNDINAMARCA' 'GUAVIARE' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META'
 'NARIÑO' 'NORTE SANTANDER' 'PUTUMAYO' 'QUINDIO' 'RISARALDA' 'SAN ANDRES'
 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE' 'VAUPES']
E_VALORMATRICULAUNIVERSIDAD ['Entre 1 millón y menos de 2.5 millones'
 'Entre 2.5 millones y menos de 4 millones'
 'Entre 4 millones y menos de 5.5 millones'
 'Entre 5.5 millones y menos de 7 millones'
 'Entre 500 mil y menos de 1 millón' 'Menos de 500 mil'
 'Más de 7 millones' 'No pagó matrícula' 'no info']
E_HORASSEMANATRABAJA ['0' 'Entre 11 y 20 horas' 'Entre 21 y 30 horas' 'Menos de 10 horas'
 'Más de 30 horas' 'no info']
F_ESTRATOVIVIENDA ['Estrato 1' 'Estrato 2' 'Estrato 3' 'Estrato 4' 'Estrato 5' 'Estrato 6'
 'Sin Estrato' 'no info']
F_TIENEINTERNET ['No' 'Si' 'no info']
F_EDUCACIONPADRE ['Educación profesional completa' 'Educación profes

### Mapear columnas binarias

Transformar las columnas con los valores `Si, No, no info` en `1, 0, -1` respectivamente.

In [160]:
binary_map = { 'Si': 1.0, 'S': 1.0, 'N': 0.0, 'No': 0.0, 'no info': -1.0 }
binary_cols = [col for col in train_df.select_dtypes(include="object").columns if train_df[col].nunique() <= 3]

print(binary_cols)

for col in binary_cols:
  train_df[col] = train_df[col].map(binary_map)

['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR']


### Mapear `E_VALORMATRICULAUNIVERSIDAD`

In [161]:
dict_map = {'Entre 1 millón y menos de 2.5 millones': 1.75,
    'Entre 2.5 millones y menos de 4 millones': 3.25,
    'Entre 4 millones y menos de 5.5 millones': 4.75,
    'Entre 5.5 millones y menos de 7 millones': 6.25,
    'Entre 500 mil y menos de 1 millón': 0.75,
    'Menos de 500 mil': 0.25,
    'Más de 7 millones': 7.75,
    'No pagó matrícula': 0.0,
    'no info': -1.0}

train_df['E_VALORMATRICULAUNIVERSIDAD'] = train_df['E_VALORMATRICULAUNIVERSIDAD'].map(dict_map)

### Mapear `F_ESTRATOVIVIENDA`

In [162]:
dict_map = {
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    'Sin Estrato': 0,
    'no info': -1}

train_df['F_ESTRATOVIVIENDA'] = train_df['F_ESTRATOVIVIENDA'].map(dict_map)

### Mapear `E_HORASSEMANATRABAJA`

In [163]:
dict_map = {
    '0': 0.0,
    'Entre 11 y 20 horas': 15.0,
    'Entre 21 y 30 horas': 25.0,
    'Menos de 10 horas': 5.0,
    'Más de 30 horas': 35.0,
    'no info': -1.0}

train_df['E_HORASSEMANATRABAJA'] = train_df['E_HORASSEMANATRABAJA'].map(dict_map)

### Mapear `F_EDUCACIONPADRE`, `F_EDUCACIONMADRE` y `E_PRGM_DEPARTAMENTO` a **one hot encoding**

Vamos a usar one hot encoding para transformar los valores, teniendo en cuenta que estas tres columnas no deberían tener un sentido de orden.
Primero vamos a unificar los valores `No aplica` y `No sabe` de las dos primeras columnas en `no info` para que no hayan muchas columnas adicionales.

In [164]:
train_df['F_EDUCACIONMADRE'] = ['no info' if val in ['No sabe', 'No Aplica'] else val for val in train_df['F_EDUCACIONMADRE'].values]
train_df['F_EDUCACIONPADRE'] = ['no info' if val in ['No sabe', 'No Aplica'] else val for val in train_df['F_EDUCACIONPADRE'].values]

Ahora aplicamos el one hot encoding

In [165]:
train_df = pd.get_dummies(train_df, columns=['F_EDUCACIONMADRE', 'F_EDUCACIONPADRE', 'E_PRGM_DEPARTAMENTO'])
train_df.columns

Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_TIENELAVADORA',
       'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO',
       'F_TIENECOMPUTADOR', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4',
       'F_EDUCACIONMADRE_Educación profesional completa',
       'F_EDUCACIONMADRE_Educación profesional incompleta',
       'F_EDUCACIONMADRE_Ninguno', 'F_EDUCACIONMADRE_Postgrado',
       'F_EDUCACIONMADRE_Primaria completa',
       'F_EDUCACIONMADRE_Primaria incompleta',
       'F_EDUCACIONMADRE_Secundaria (Bachillerato) completa',
       'F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta',
       'F_EDUCACIONMADRE_Técnica o tecnológica completa',
       'F_EDUCACIONMADRE_Técnica o tecnológica incompleta',
       'F_EDUCACIONMADRE_no info',
       'F_EDUCACIONPADRE_Educación profesional completa',
       'F_EDUCA

### Mapear `E_PRGM_ACADEMICO`

Esta columna tiene una cardinalidad de 948, por lo que hacer un one hot encoding añadiría muchas columnas al dataset. Es más viable usar otro método de transformación de los datos. Por lo que vamos a usar **label encoding**.

In [166]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['E_PRGM_ACADEMICO'] = le.fit_transform(train_df['E_PRGM_ACADEMICO'])

### Mapear la variable objetivo `RENDIMIENTO_GLOBAL`

In [167]:
dict_map = {
    'alto': 3,
    'bajo': 0,
    'medio-alto': 2,
    'medio-bajo': 1}

train_df['RENDIMIENTO_GLOBAL'] = train_df['RENDIMIENTO_GLOBAL'].map(dict_map)

### Resultado final

Todas las columnas fueron procesadas y convertidas a valores numericos que los modelos de aprendizaje automatico puedan entender.

In [172]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 69 columns):
 #   Column                                                 Non-Null Count   Dtype  
---  ------                                                 --------------   -----  
 0   PERIODO_ACADEMICO                                      692500 non-null  int64  
 1   E_PRGM_ACADEMICO                                       692500 non-null  int64  
 2   E_VALORMATRICULAUNIVERSIDAD                            692500 non-null  float64
 3   E_HORASSEMANATRABAJA                                   692500 non-null  float64
 4   F_ESTRATOVIVIENDA                                      692500 non-null  int64  
 5   F_TIENEINTERNET                                        692500 non-null  float64
 6   F_TIENELAVADORA                                        692500 non-null  float64
 7   F_TIENEAUTOMOVIL                                       692500 non-null  float64
 8   E_PRIVADO_LIBERTAD                

In [171]:
train_df.head(10)

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,...,E_PRGM_DEPARTAMENTO_NORTE SANTANDER,E_PRGM_DEPARTAMENTO_PUTUMAYO,E_PRGM_DEPARTAMENTO_QUINDIO,E_PRGM_DEPARTAMENTO_RISARALDA,E_PRGM_DEPARTAMENTO_SAN ANDRES,E_PRGM_DEPARTAMENTO_SANTANDER,E_PRGM_DEPARTAMENTO_SUCRE,E_PRGM_DEPARTAMENTO_TOLIMA,E_PRGM_DEPARTAMENTO_VALLE,E_PRGM_DEPARTAMENTO_VAUPES
0,20212,300,6.25,5.0,3,1.0,1.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,20212,249,3.25,0.0,3,0.0,1.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,20203,819,3.25,35.0,3,1.0,1.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,20195,14,4.75,0.0,4,1.0,1.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
4,20212,907,3.25,25.0,3,1.0,1.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5,20203,811,7.75,5.0,5,1.0,1.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
6,20183,455,3.25,25.0,2,1.0,1.0,1.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
7,20183,94,1.75,15.0,2,1.0,1.0,0.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
8,20212,452,6.25,5.0,1,1.0,1.0,1.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
9,20183,14,3.25,35.0,5,1.0,1.0,1.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False


### Guardar el resultado

In [173]:
train_df.to_csv('train_preprocessed.csv', index=False)